<a href="https://colab.research.google.com/github/stephenbaek/dlcourse/blob/main/notebooks/L02_Cats_and_Dogs/simple_classifier.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# Cats and Dogs: Building a simple classifier using TensorFlow

In the previous session, we have learned some basics of TensorFlow, namely `tf.Tensor`s and their operations. In this session, we are going to build our first machine learning model using TensorFlow. The model we are going to build is a simple softmax classifier that takes an image as an input and returns class probabilities (see Lecture 2 for details). To train and test our model, we will use the cats and dogs dataset (https://www.microsoft.com/en-us/download/details.aspx?id=54765).

## Download Data

Always, the very first step to develop a machine learning model is to create a pipeline to read data. To get started, let us first download the cats and dogs dataset. I made a zip file available to download directly (the original distribution can be found [here](https://www.microsoft.com/en-us/download/details.aspx?id=54765)). Just run the cell below to download and unzip the dataset.

In [ ]:
import urllib.request
from zipfile import ZipFile
import os

if not os.path.exists('cats_and_dogs.zip'):
    # download 'cats_and_dogs.zip' file from the URL. this may take several minutes.
    urllib.request.urlretrieve(
        32,
        "cats_and_dogs.zip"
    )

    # unzip the dataset
    with ZipFile('cats_and_dogs.zip', 'r') as zip:
        zip.extractall()

## Read and Preprocess Images

The data set we have just downloaded are structured in the following way:

```bash
|- PetImages             # image root
|    |- Cat              # subdirectory for cat images
|        |- 0.jpg
|        |- 1.jpg
|        |- 2.jpg
|        |- ...
|    |- Dog              # subdirectory for dog images
|        |- 0.jpg
|        |- 1.jpg
|        |- 2.jpg
|        |- ...
|- readme[1].txt         # readme file
|- MSR-LA - 3467.docx    # data set documentation
```

As you can see, images are structured in a way that images of different classes are under separate subfolders. Below shows a simple example of how you could list all files in each subfolder and count the number of images.

In [ ]:
import pathlib      # pathlib is a useful tool for listing all files in a folder
data_dir = pathlib.Path('PetImages')    # create a `Path` object for the master folder
cat_dir = data_dir / 'Cat'              # `Path` object for 'PetImages/Cat' folder. operator '/' is used to add sub-folders.
dog_dir = data_dir / 'Dog'              # `Path` object for 'PetImages/Dog' folder.

In [ ]:
cat_files = list(cat_dir.glob('*.jpg'))    # list all files with an extension .jpg
dog_files = list(dog_dir.glob('*.jpg'))

print('Cat Images:', len(cat_files))
print('Dog Images:', len(dog_files))

Further, just to see how those images look like, you can use Python Imaging Library (PIL):

In [ ]:
import PIL
import PIL.Image
PIL.Image.open(str(cat_files[0]))

## TensorFlow Data Pipeline

While reading images using Pathlib and PIL should be enough for you to get things done, here I show you a little more advanced (but yet more convenient) version for pipelining input images. TensorFlow comes with a `Dataset` module for optimizing the process to [extract, transform, and load (ETL)](https://en.wikipedia.org/wiki/Extract,_transform,_load) data during training ML models. While it should be unnecessary to deep dive into ETL optimization, here's an example of how you can read a `tf.Dataset` from a directory:

In [ ]:
import tensorflow as tf

image_size = (64, 64)   # resize images to (256, 256)
batch_size = 32           # "bite-size" chunks

# train dataset
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    subset='training',
    validation_split=0.3,
    seed=1,
)

# test dataset
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    subset='validation',
    validation_split=0.3,
    seed=1,
)

So, there's a lot to unpack here. First of all, `tf.keras.preprocessing.image_dataset_from_directory()` is a helper function to create a `tf.Dataset` object from a directory. What it does is simply to list all the image files in `data_dir` and get prepared for reading pixel values. Here's an important thing to note: creating a `tf.Dataset()` doesn't mean that you are reading all images from the hard drive and mount them on a RAM. Instead, at the time of creation, it doesn't actually read anything from the hard drive, but later, only when there is a request to fetch pixel values, it then reads the corresponding image from the hard drive and spit out the pixel values (lazy loading). The reason for this is simple: if you have a large image dataset (e.g. ImageNet) that has millions of images, it is simply impossible to physically load all of the pixel values to a RAM. Instead, it would be more realistic to just let them stay in the hard drive and fetch a few of them, only when needed.

`tf.Dataset` actually goes one step further than that. Since usually loading images from a hard drive takes a lot of time, it often becomes a bottleneck during training. For example, let say you are dividing millions of images into small, "bite-size" subsets, say 32 images per batch (btw, this is what `batch_size` in the code above was doing), and feed them into a ML model. A naive process will look something like: (1) read a batch of 32 images, (2) train a model with the batch just loaded, (3) read the next batch of 32 images, (4) train the model with the new batch, (5) read the next batch, (6) train, so on and so forth. Now, with GPU accelerators, usually the training steps won't take a terribly large amount of time. However, accessing a hard drive and reading files from there takes much longer time than the training steps. So from the GPU processor's perspective, you consume a small set of images for training, wait for a long time for the next batch to arrive, consume the new batch again, wait for another long time for the next batch, and so on. This renders a great amount of time in between actual training steps where you "starve" for a new batch of data. `tf.Dataset` solves this issue quite nicely by introducing the concept of *prefetch*. What it does is simply to transform the sequence of 'read-train-read-train-read-train-...' to a parallel run, such that, while GPU is training a model with the current batch of images, CPU in parallel reads the next batch of image files and get ready to toss them to GPU, immediately after GPU finishes the current training step. This can significantly accelerate the speed of training an ML model, especially when you have a large image dataset. If you are interested in learning more about it, here is [an official documentation from TensorFlow](https://www.tensorflow.org/guide/data_performance) where you can find more detailed descriptions. For now, let's not worry about optimizing the ETL pipeline too much. For the scope of this session, it should be enough to understand that there's something like this at a high-level.

Now that we know what `tf.keras.preprocessing.image_dataset_from_directory()` method does, here are some optional arguments you can specify. First, the `labels='inferred'` option implies that the class labels (cat vs dog in our example) will be inferred from folder names, which can come quite handy for image classification tasks. The `label_mode='int'` option lets you convert the string labels `'cat'` and `'dog'` to numeric labels `0` and `1`, which is a necessary step in almost all cases. The other options, such as `color_mode='rgb'`, `batch_size=batch_size`, `image_size=image_size`, and `shuffle=True`, are quite self-explanatory, so I'll just leave a link to the [TensorFlow documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory) here instead.

One thing that is still worthwhile to mention is the `validation_split=0.3` option. The `tf.keras.preprocessing.image_dataset_from_directory()` method in fact allows you to set aside a subset of your data for testing your algorithm in the future. `validation_split` option specifies the portion of such set-aside subset. When `validation_split` option is passed, you are also required to pass `subset` option together, which can be either `'traiing'` or `'validation'`. If `subset='validation'`, then `tf.keras.preprocessing.image_dataset_from_directory()` will return the set-aside subset. Otherwise, `tf.keras.preprocessing.image_dataset_from_directory()` will return the rest of the dataset. Note here that the `seed` value must be exactly the same when you create a training dataset and a set-aside subset. The `seed` value determines how the original dataset is going to be shuffled before to split train-test ([pseudo-random numbers](https://www.random.org/randomness/)). If you use different `seed`, you will end up shuffling the original dataset differently, each time you split training and testing set, resulting in the leak of test data into the training set. Hence, you need to be very careful not to commit such a mistake.

## Fetching a Batch of Data from `tf.Dataset`

While it required a quite a bit of background knowledge to understand `tf.Dataset`, the actual usage is rather quite simple. First of all, you can take a few batches from the dataset using `take()` method:

In [ ]:
zero_batches = train_ds.take(0)    # take zero batches from the dataset (empty batch)
one_batch = train_ds.take(1)       # take one batch
two_batches = train_ds.take(2)     # take two batches

print(zero_batches)
print(one_batch)
print(two_batches)

Again, taking a batch doesn't mean that you are actually reading files from the hard drive, which is why you don't see any of their contents when we printed batches above. To see what's actually in the batch, we need to request for the actual values. While you don't have to memorize this, here's one trick to do so:

In [ ]:
zero_batches_values = list(zero_batches.as_numpy_iterator())
one_batch_values = list(one_batch.as_numpy_iterator())
two_batches_values = list(two_batches.as_numpy_iterator())
print(len(zero_batches_values))
print(len(one_batch_values))
print(len(two_batches_values))

Again, you don't have to memorize or worry about the `as_numpy_iterator()` method. We are not going to use it later. The above lines are just to explain what happens in `tf.Dataset` objects. 

Now, let's examine the contents (values) in the batches. Apparently, `zero_batches_values` is empty, as we can guess from its length. `one_batch_values` seems to contain one item in the list, whereas `two_batches_values` seems to contain two items in the list, as expected. Let's dissect them to see what's in there:

In [ ]:
print(len(one_batch_values[0]))
print(len(two_batches_values[0]))
print(len(two_batches_values[1]))

First of all, it seems like each batch seems to contain 2 chunks of information. They are in fact an image tensor and a label tensor, as we can observe from below:

In [ ]:
print(one_batch_values[0][0].shape)
print(one_batch_values[0][1].shape)
print(two_batches_values[0][0].shape)
print(two_batches_values[0][1].shape)
print(two_batches_values[1][0].shape)
print(two_batches_values[1][1].shape)

Note that the first element in the batch, i.e. an image tensor, has a shape of `(32, 256, 256, 3)`, thus a rank 4 tensor. 32 is `batch_size` we defined above, and `(256, 256)` equals to `image_size`. The last dimension `3` comes from the RGB representation of the images.

The second element in the batch, i.e. a label tensor, has a shape of `(32,)`, a rank 1 tensor. The elements in this rank 1 tensor are essentially numeric labels corresponding to cat/dog:

In [ ]:
print(one_batch_values[0][1])
print(two_batches_values[0][1])
print(two_batches_values[1][1])

So we have a bunch of zeros and ones, indicating either cat or dog, but in which order? You can check this by calling `dataset.class_names`:

In [ ]:
class_names = train_ds.class_names
print(class_names)

So, from this, we can see that `0` means cat and `1` means dog.

Now, one thing worthwhile to mention before moving on is that the contents of `one_batch_values[0][1]` and `two_batches_values[0][1]` are not the same. This is an important thing to notice because it means that each time you call `dataset.take()`, it will fetch a new batch. So in our code above, when we called `train_ds.take(1)` it retruned the very first batch in the dataset, namely batch id #0, and later we called `train_ds.take(2)`, it returned the next two batches, namely batch ids #1 and #2, as opposed to #0 and #1.

One final note here is that `tf.Dataset` objects are iterable, so that you can use a for-loop to traverse through their contents:

In [ ]:
for image_batch, labels_batch in test_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break     # break out of the for loop after the first iteration. just for demo.

## Display Images

Here is a quick example for displaying train images using [`matplotlib`](https://matplotlib.org/), just for a sanity check. `Matplotlib` provides a variety of predefined functions for creating and displaying scientific plots. The functions look quite similar to MATLAB, so if you have previous experience with MATLAB, they should not be too foreign. In this class, we will use `imshow` quite frequently, so it would be useful to remember the following routine:
```python
import matplotlib.pyplot as plt   # import matplotlib
plt.figure()                      # create a new figure
plt.imshow(img)                   # display an image
plt.show()                        # render it
```

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## Data Normalization & Preprocessing

Another benefit of using `tf.Dataset` is that it allows efficient pipelining of data transformation procedure. To this end, `tf.Dataset.map()` can come in handy. `tf.Dataset.map()` is a method that allows you to apply a function (transformation) to every single element of your dataset. For example, the code below converts the raw dataset in color (RGB) images to grayscale:

In [ ]:
# function to be applied to each element in a dataset
def convert_to_gray(image, label):    # note each element is comprised of an image and a label
    return tf.reduce_mean(image, axis=-1), label

gray_train_ds = train_ds.map(convert_to_gray)
gray_test_ds = test_ds.map(convert_to_gray)

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in gray_train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"), cmap='gray')
        plt.title(class_names[labels[i]])
        plt.axis("off")

Similarly, you can rescale pixel values from $[0, 255]$ to $[0, 1]$:

In [ ]:
def rescale(image, label):
    return image / 255.0, tf.one_hot(label,2)

normalized_train_ds = gray_train_ds.map(rescale)
normalized_test_ds = gray_test_ds.map(rescale)

## Training a simple classifier

Finally, we are ready to train a simple image classifier using TensorFlow. While discussions on the details of building a classifier will be deferred to the next lab session, below is a minimalistic code to build, complile, and train a logistic regression model. Observe how accuracy evolves as the iteration continues. Again, try not to focus to much on the details---we will deep dive into them in the next lab session.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(2, activation='softmax')
    ])

model.compile(loss='binary_crossentropy', metrics=['accuracy'])
model.fit(normalized_train_ds, epochs=10)

## Conclusion

Congratulations! You have just built and run your first TensorFlow image classifier successfully. In terms of the accuracy, the classifier only does  slightly better than a random guess, but I would call it a great first step to the world of machine learning and computer vision.

If this was your first time doing some serious Python programming, I strongly recommend you scroll up again and walk through the tutorial carefully one more time.